In [1]:
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element
import pickle
import os
import pandas as pd
import numpy as np
import nltk.data
from gensim.models import Word2Vec
import multiprocessing
from copy import deepcopy
import datetime
from sklearn.metrics import roc_auc_score

In [2]:
#on VM
os.chdir('/mnt/mypartition/Desktop2/pubmed_nlp_research/DeepLearningMovies_datasets/')
import KaggleWord2VecUtility

In [3]:
#on local:
import sys
sys.path.append('/home/sarahwie/Documents/pubmed-nlp-research/DeepLearningMovies_datasets/')
from KaggleWord2VecUtility import KaggleWord2VecUtility
DATADIR='/home/sarahwie/Documents/pubmed-nlp-research/DeepLearningMovies_datasets/'

In [3]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

### Checking that XML input has correct journals and in correct format (with abstracts and no Unicode)

In [10]:
#ignore for now
# def combine_xml(files):
#     first = None
#     for filename in files:
#         data = ET.parse(filename).getroot()
#         if first is None:
#             first = data
#         else:
#             first.extend(data)
#     if first is not None:
#         return first

# a = datetime.datetime.now().replace(microsecond=0)
  
# #replace these URLs!
# #rootComb = combine_xml(['/home/sarahwie/Documents/pubmed_result_clinical.xml','/home/sarahwie/Documents/pubmed_result_basic_science.xml'])
# rootComb = combine_xml(['/mnt/mypartition/pubmed_result_clinical.xml','/mnt/mypartition/pubmed_result_basic_science.xml'])

# b = datetime.datetime.now().replace(microsecond=0)
# print(b-a)

In [ ]:
#print len(rootComb.getchildren())

This code subsets the records down to those with abstracts and checks that correct journal names have been queried.

### Uses too much memory- don't use now that we know all records have correct title and abstracts.

In [31]:
#make new XML file
rootNew = Element('MedlineCitationSet')
children = []

In [32]:
titles = ['Journal of general internal medicine', 'Annals of emergency medicine', 'Pediatrics', 
                  'The Journal of pediatrics', 'Journal of clinical hypertension (Greenwich, Conn.)', 'Annals of medicine', 
                  'Academic emergency medicine : official journal of the Society for Academic Emergency Medicine', 
                  'Annals of internal medicine','The Journal of biological chemistry','Nucleic acids research', 'Cell', 
                      'Methods in molecular biology (Clifton, N.J.)', 'Nature cell biology', 
                      'Progress in biophysics and molecular biology', 'Molecular and cellular biology']


In [6]:
# a = datetime.datetime.now().replace(microsecond=0)

# #replace these URLs!
# filenames = ['/mnt/mypartition/pubmed_result_clinical.xml','/mnt/mypartition/pubmed_result_basic_science.xml']   

# i = 0
# #read in files
# j = 0

# xml_file = ET.parse(file)
# root = xml_file.getroot()
# for record in root.findall('PubmedArticle'):
#     title = record.find('MedlineCitation').find('Article').find('Journal').find('Title').text
#     #check that all title names match list
#     if title in titles:
#         try:
#             abstract = record.find('MedlineCitation').find('Article').find('Abstract').find('AbstractText').text
#             #append to subset file
#             children.append(record)
#             i = i + 1
#         except AttributeError:
#             #dont include if no abstract
#             print 'no abstract!'

#     else: 
#         print title
#         j = j+1

# #print num records obtained (should be 44717)
# print i
# #print num. records with wrong titles
# print j
# b = datetime.datetime.now().replace(microsecond=0)
# print(b-a)

/mnt/mypartition/pubmed_result_clinical.xml
/mnt/mypartition/pubmed_result_basic_science.xml
44717
0
0:02:27


In [ ]:
# a = datetime.datetime.now().replace(microsecond=0)

# rootNew.extend(children)

# #create tree and write to file
# #tree = ET.ElementTree(rootNew)
        
# pickle.dump(rootNew, open('/mnt/mypartition/Desktop2/rootJournalsSubset.p', 'wb'))

# b = datetime.datetime.now().replace(microsecond=0)
# print(b-a)

In [ ]:
#size of new subset- should be 44717
print len(rootNew.getchildren())
#verify that this is the correct number: yes

### *New plan: run the below separately for both files and then append the pandas dataframes in memory.

### Convert XML to Pandas dataframes (include title, abstract, mesh terms, label)

In [13]:
titlesClinical = ['Journal of general internal medicine', 'Annals of emergency medicine', 'Pediatrics', 
                  'The Journal of pediatrics', 'Journal of clinical hypertension (Greenwich, Conn.)', 'Annals of medicine', 
                  'Academic emergency medicine : official journal of the Society for Academic Emergency Medicine', 
                  'Annals of internal medicine']

titlesBasicScience = ['The Journal of biological chemistry','Nucleic acids research', 'Cell', 
                      'Methods in molecular biology (Clifton, N.J.)', 'Nature cell biology', 
                      'Progress in biophysics and molecular biology', 'Molecular and cellular biology']

In [14]:
def iter_docs(author):
    author_attr = author.attrib
    for record in author.findall('PubmedArticle'):
        doc_dict = author_attr.copy()

        title = record.find('MedlineCitation').find('Article').find('ArticleTitle').text
        doc_dict['title'] = title
        journal = record.find('MedlineCitation').find('Article').find('Journal').find('Title').text
        doc_dict['journal'] = journal
        
        abstract = ''
        for abst in record.find('MedlineCitation').find('Article').find('Abstract').findall('AbstractText'):
            if abst.text != None:
                if type(abst.text) != str:
                    abst.text = abst.text.encode('utf8')
                abstract = abstract + abst.text
        doc_dict['abstract'] = abstract
        
        #get MeSH terms as strings to simplify later processing
        meshDescriptor = ''
        meshQualifier = ''
        if record.find('MedlineCitation').find('MeshHeadingList') is not None:
            for mesh in record.find('MedlineCitation').find('MeshHeadingList').findall('MeshHeading'):
                if mesh.find('DescriptorName') is not None:
                    for desc in mesh.findall('DescriptorName'):
                        if meshDescriptor == '':
                            meshDescriptor = desc.text
                        else:
                            meshDescriptor = meshDescriptor + ' ' + desc.text
                if mesh.find('QualifierName') is not None:
                    for qual in mesh.findall('QualifierName'):
                        if meshQualifier == '':
                            meshQualifier = qual.text
                        else:
                            meshQualifier = meshQualifier + ' ' + qual.text
        doc_dict['qualifier_terms'] = meshQualifier
        doc_dict['descriptor_terms'] = meshDescriptor
        if journal in titlesClinical:
            doc_dict['sentiment'] = 0
        elif journal in titlesBasicScience:
            doc_dict['sentiment'] = 1
        else:
            print 'title not in lists'
            print journal
        yield doc_dict
        
#df = pd.DataFrame(list(iter_docs(rootNew)))
#df = df[['sentiment', 'journal', 'title', 'abstract', 'qualifier_terms', 'descriptor_terms']]
#print df

#shuffle rows- necessary??
#df = df.iloc[np.random.permutation(len(df))]
#df = df.reset_index(drop=True)
#print df

In [15]:
a = datetime.datetime.now().replace(microsecond=0)

filenames = ['/mnt/mypartition/pubmed_result_clinical.xml','/mnt/mypartition/pubmed_result_basic_science.xml']   

i = 0
for file in filenames:
    print file
    xml_file = ET.parse(file)
    root = xml_file.getroot()
    if i == 0:
        df1 = pd.DataFrame(list(iter_docs(root)))
        df1 = df1[['sentiment', 'journal', 'title', 'abstract', 'qualifier_terms', 'descriptor_terms']]
    elif i == 1:
        df2 = pd.DataFrame(list(iter_docs(root)))
        df2 = df2[['sentiment', 'journal', 'title', 'abstract', 'qualifier_terms', 'descriptor_terms']]
    i = i + 1
    
b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

/mnt/mypartition/pubmed_result_clinical.xml
Index([u'sentiment', u'journal', u'title', u'abstract', u'qualifier_terms',
       u'descriptor_terms'],
      dtype='object')
/mnt/mypartition/pubmed_result_basic_science.xml
Index([u'sentiment', u'journal', u'title', u'abstract', u'qualifier_terms',
       u'descriptor_terms'],
      dtype='object')
0:02:36


In [22]:
print df1.shape
print df2.shape

(19826, 6)
(24891, 6)


In [18]:
df1[0:5]

sentiment                                            journal  \
0          0  Academic emergency medicine : official journal...   
1          0  Academic emergency medicine : official journal...   
2          0  Academic emergency medicine : official journal...   
3          0  Academic emergency medicine : official journal...   
4          0  Academic emergency medicine : official journal...   

                                               title  \
0  Accuracy of Point-of-Care Lung Ultrasonography...   
1  Patient and process factors associated with ty...   
2  Comparative Trends and Downstream Outcomes of ...   
3                     The Man in Hospital Room 1233.   
4  Extravasation Risk Using Ultrasound Guided Per...   

                                            abstract qualifier_terms  \
0  To determine the test performance characterist...                   
1  In-hospital factors contribute more to delayed...                   
2  Coronary computerized tomography angiography (...                   
3  As I got off the elevator approaching my uncle...                   
4  Ultrasound guided intravenous catheter (USGIV)...                   

  descriptor_terms  
0                   
1                   
2                   
3                   
4

In [20]:
df2[0:5]

sentiment journal                                              title  \
0          1    Cell     SnapShot: Mechanical Forces in Development II.   
1          1    Cell  Real-Time Imaging of Translation on Single mRN...   
2          1    Cell  Dynamics of Translation of Single mRNA Molecul...   
3          1    Cell  Lung Adenocarcinoma Distally Rewires Hepatic C...   
4          1    Cell   Temporal Stability of the Human Skin Microbiome.   

                                            abstract qualifier_terms  \
0  The forces shaping an organism are not exclusi...                   
1  Translation is under tight spatial and tempora...                   
2  Regulation of mRNA translation, the process by...                   
3  The circadian clock controls metabolic and phy...                   
4  Biogeography and individuality shape the struc...                   

  descriptor_terms  
0                   
1                   
2                   
3                   
4

### Concatenate pandas dataframes

In [23]:
frames = [df1, df2]
dfJournalsSubset = pd.concat(frames, ignore_index=True)

0:00:00


In [26]:
print dfJournalsSubset.shape
dfJournalsSubset[19824:19830]

(44717, 6)


sentiment     journal  \
19824          0  Pediatrics   
19825          0  Pediatrics   
19826          1        Cell   
19827          1        Cell   
19828          1        Cell   
19829          1        Cell   

                                                   title  \
19824  Intracranial hemorrhage as the initial manifes...   
19825  Immunizations for patients with metabolic diso...   
19826     SnapShot: Mechanical Forces in Development II.   
19827  Real-Time Imaging of Translation on Single mRN...   
19828  Dynamics of Translation of Single mRNA Molecul...   
19829  Lung Adenocarcinoma Distally Rewires Hepatic C...   

                                                abstract  \
19824  Intracranial hemorrhage in a term neonate is a...   
19825  Individuals with underlying metabolic disorder...   
19826  The forces shaping an organism are not exclusi...   
19827  Translation is under tight spatial and tempora...   
19828  Regulation of mRNA translation, the process by...   
19829  The circadian clock controls metabolic and phy...   

                                         qualifier_terms  \
19824  etiology pathology blood complications diagnos...   
19825  immunology therapy immunology therapy contrain...   
19826                                                      
19827                                                      
19828                                                      
19829                                                      

                                        descriptor_terms  
19824  Apnea Brain Carbohydrate Metabolism, Inborn Er...  
19825  Adolescent Amino Acid Metabolism, Inborn Error...  
19826                                                     
19827                                                     
19828                                                     
19829

In [28]:
pickle.dump(dfJournalsSubset, open('/mnt/mypartition/Desktop2/pubmed_nlp_research/pickled_objects/dfJournalsSubset.p', 'wb'))

### Word2Vec Inversion with 5-fold Cross Validation

In [31]:
df = pickle.load(open('/mnt/mypartition/Desktop2/pubmed_nlp_research/pickled_objects/dfJournalsSubset.p', 'rb'))
print len(df)

44717


In [32]:
"""
docprob takes two lists
* docs: a list of documents, each of which is a list of sentences
* models: the candidate word2vec models (each potential class)

it returns the array of class probabilities.  Everything is done in-memory.
"""

import pandas as pd # for quick summing within doc

def docprob(docs, mods):
    # score() takes a list [s] of sentences here; could also be a sentence generator
    sentlist = [s for d in docs for s in d]
    # the log likelihood of each sentence in this review under each w2v representation
    llhd = np.array( [ m.score(sentlist, len(sentlist)) for m in mods ] )
    # now exponentiate to get likelihoods, 
    lhd = np.exp(llhd - llhd.max(axis=0)) # subtract row max to avoid numeric overload
    # normalize across models (stars) to get sentence-star probabilities
    #all this transposing business does is make it so the total probability of a word 
    #   equals 1 between the 2 arrays (positive prob and negative).
    #and the pandas data frame just puts everything into rows/columns format for easy viz
    prob = pd.DataFrame( (lhd/lhd.sum(axis=0)).transpose() )
    # and finally average the sentence probabilities to get the review probability
    prob["doc"] = [i for i,d in enumerate(docs) for s in d]
    prob = prob.groupby("doc").mean()
    return prob

In [55]:
a = datetime.datetime.now().replace(microsecond=0)

#5-fold stratified cross validation
from sklearn.cross_validation import StratifiedKFold

y = df['sentiment'].values
skf = StratifiedKFold(y, n_folds=5, shuffle=True)

i = 1
avg = []
for train_index, test_index in skf:
    
    print("ROUND", i)
    i = i + 1
    
    #use the indexes to subset the df pandas dataframe
    train1, test1 = df.iloc[train_index], df.iloc[test_index]

    print len(np.where(train1['sentiment'] == 0)[0])
    print len(np.where(train1['sentiment'] == 1)[0])
    print len(np.where(test1['sentiment'] == 0)[0])
    print len(np.where(test1['sentiment'] == 1)[0])

    # ****** Split the training set into clean sentences
    #
    sentences_pos = []  # Initialize an empty list of sentences
    sentences_neg = []  # Initialize an empty list of sentences

    #here change to include all journal name labels of positive and negative
    inxs_pos = np.where(train1['sentiment'] == 1)[0].tolist()
    inxs_neg = np.where(train1['sentiment'] == 0)[0].tolist()

    print "Parsing sentences from training set"
    for inx in inxs_pos:
        review = train1["abstract"].iloc[inx]
        title = train1['title'].iloc[inx]
        descriptors = train1['descriptor_terms'].iloc[inx]
        qualifiers = train1['qualifier_terms'].iloc[inx]
        sentences_pos += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
        sentences_pos += KaggleWord2VecUtility.review_to_sentences(title, tokenizer)
        sentences_pos += KaggleWord2VecUtility.review_to_sentences(descriptors, tokenizer)         
        sentences_pos += KaggleWord2VecUtility.review_to_sentences(qualifiers, tokenizer)       
        
        
    for inx in inxs_neg:
        review = train1["abstract"].iloc[inx]
        title = train1['title'].iloc[inx]
        descriptors = train1['descriptor_terms'].iloc[inx]
        qualifiers = train1['qualifier_terms'].iloc[inx]
        sentences_neg += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
        sentences_neg += KaggleWord2VecUtility.review_to_sentences(title, tokenizer)
        sentences_neg += KaggleWord2VecUtility.review_to_sentences(descriptors, tokenizer)         
        sentences_neg += KaggleWord2VecUtility.review_to_sentences(qualifiers, tokenizer)       
        
    # ****** Split the labeled and unlabeled training sets into clean sentences
    #
    sentences = []  # Initialize an empty list of sentences

    for i in range(len(train1)):
        review = train1['abstract'][i]
        title = train1['title'][i]
        descriptors = train1['descriptor_terms'][i]
        qualifiers = train1['qualifier_terms'][i]
        sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
        sentences += KaggleWord2VecUtility.review_to_sentences(title, tokenizer)
        sentences += KaggleWord2VecUtility.review_to_sentences(descriptors, tokenizer)
        sentences += KaggleWord2VecUtility.review_to_sentences(qualifiers, tokenizer)


    print "Building and training w2v models"
    ## create a w2v learner 
    basemodel = Word2Vec(
        workers=multiprocessing.cpu_count(), # use your cores
        iter=3, # iter = sweeps of SGD through the data; more is better
        hs=1, negative=0 # we only have scoring for the hierarchical softmax setup
        )
    basemodel.build_vocab(sentences) 

    #train models
    models = [deepcopy(basemodel) for y in range(2)]
    models[0].train(sentences_neg, total_examples=len(sentences_neg) )
    models[1].train(sentences_pos, total_examples=len(sentences_pos) )

    print "Parsing test sentences"
    # read in the test set as a list of a list of words
    docs = []
    i = 0
    for review in test1["abstract"]:
        if i == 0:
            docs.append(KaggleWord2VecUtility.review_to_sentences(review, tokenizer))


    print "scoring test set"
    # get the probs (note we give docprob our test set plus the models)
    probs = docprob(docs,models).astype(object)

    predictions = np.ones((probs.shape[0]))

    #print test1.loc[0:50,['sentiment','abstract']]
    #print probs[0:10]

    predictions[np.where(probs.iloc[:,0] > 0.5)] = 0 # The first column is the negative model
    print predictions

    #think these line up
    #ok so issue is just that document probabilities are so much higher scored for 0 than 1 for some reason.
    #Cant figure out why...most likely not a bug but just the word2vec model not performing great
    score = roc_auc_score(test1["sentiment"], predictions)
    #score = np.size(np.where(predictions == test1["sentiment"]))*1./np.size(predictions)
    print score
    
b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

('ROUND', 1)
161
176
41
44
Parsing sentences from training set
Building and training w2v models
Parsing test sentences
scoring test set
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
85
0.5
0:00:02


In [9]:
a = datetime.datetime.now().replace(microsecond=0)

#5-fold stratified cross validation
from sklearn.cross_validation import StratifiedKFold
import numpy as np

#because no validation set, 4/5 of values go to train and 1/5 to test
#is this too high?***
#even though we shuffle, not as randomly distributed as the former method was
y = df['sentiment'].values
skf = StratifiedKFold(y, n_folds=5, shuffle=True)

i = 1
avg = []
for train_index, test_index in skf:
    print("ROUND", i)
    i = i + 1
    #use the indexes to subset the df pandas dataframe
    train1, test1 = df.iloc[train_index], df.iloc[test_index]
    
    # ****** Split the training set into clean sentences
    #
    sentences_pos = []  # Initialize an empty list of sentences
    sentences_neg = []  # Initialize an empty list of sentences

    #here change to include all journal name labels of positive and negative
    inxs_pos = np.where(train1['sentiment'] == 1)[0].tolist()
    inxs_neg = np.where(train1['sentiment'] == 0)[0].tolist()

    print "Parsing sentences from training set"
    for inx in inxs_pos:
        review = train1["abstract"].iloc[inx]
        sentences_pos += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

    for inx in inxs_neg:
        review = train1["abstract"].iloc[inx]
        sentences_neg += KaggleWord2VecUtility.review_to_sentences(review, tokenizer) 
        
    # ****** Split the labeled and unlabeled training sets into clean sentences
    #
    sentences = []  # Initialize an empty list of sentences

    for review in train1["abstract"]:
        sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
    
    print "Building and training w2v models"
    ## create a w2v learner 
    basemodel = Word2Vec(
        workers=multiprocessing.cpu_count(), # use your cores
        iter=3, # iter = sweeps of SGD through the data; more is better
        hs=1, negative=0 # we only have scoring for the hierarchical softmax setup
        )
    basemodel.build_vocab(sentences) 
    
    #train models
    models = [deepcopy(basemodel) for y in range(2)]
    models[0].train(sentences_neg, total_examples=len(sentences_neg) )
    models[1].train(sentences_pos, total_examples=len(sentences_pos) )
    
    print "Parsing test sentences"
    # read in the test set as a list of a list of words
    docs = []
    for review in test1["abstract"]:
        docs.append(KaggleWord2VecUtility.review_to_sentences(review, tokenizer))
    
    print "scoring test set"
    # get the probs (note we give docprob our test set plus the models)
    probs = docprob(docs,models).astype(object)
    
    predictions = np.ones((probs.shape[0]))
    
    predictions[np.where(probs.iloc[:,0] > 0.5)] = 0 # The first column is the negative model
    print predictions
    print np.size(predictions)
    
    score = roc_auc_score(test1["sentiment"], predictions)
    #score = np.size(np.where(predictions == test1["sentiment"]))*1./np.size(predictions)
    print score
    #append to average
    avg.append(score)


print("average of 5 rotations:", sum(avg)/float(len(avg)))
    
b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

('ROUND', 1)
Parsing sentences from training set


/home/sarahwie/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Building and training w2v models
Parsing test sentences
scoring test set
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
85
0.510532150776
('ROUND', 2)
Parsing sentences from training set
Building and training w2v models
Parsing test sentences
scoring test set
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
85
0.487804878049
('ROUND', 3)
Parsing sentences from training set
Building and training w2v mo

In [25]:
#confirming index match to dataframe for the last fold
print train_index
print test_index
print y_train
print y_test

[  0   1   3   5   6   7   9  11  12  13  14  15  16  17  18  19  20  21
  22  23  25  26  27  28  29  30  32  33  34  35  36  37  38  41  42  44
  45  46  47  48  49  50  51  52  54  55  56  57  58  59  60  61  64  65
  67  68  69  70  71  72  73  74  75  76  77  78  79  82  83  84  86  87
  89  90  91  92  93  95  96  97  98  99 100 101 102 103 104 106 107 108
 109 112 113 114 116 118 119 120 121 122 123 124 125 126 127 129 130 131
 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 150 151 153
 154 155 156 157 158 159 160 161 162 164 165 166 167 168 169 170 171 172
 174 178 179 180 181 183 184 185 186 188 189 190 191 192 193 194 198 199
 202 203 204 206 207 208 209 210 212 213 214 215 216 219 220 221 223 224
 225 226 227 228 229 231 232 233 234 235 236 237 238 239 240 242 243 244
 246 247 248 249 250 251 252 253 254 255 256 257 259 260 261 262 263 264
 265 266 267 268 270 272 273 274 275 276 278 279 280 282 284 285 286 287
 288 289 290 292 293 294 295 296 297 299 300 301 30

In [27]:
#print first record in test set (note has different index value, indexed instead by location (pos. 0))
print test1.iloc[[0]]

   sentiment         journal  \
2          0  Brain research   

                                            abstract  
2  The neuranatomical location and pharmacologica...  


### Import KaggleWord2VecUtility since didn't work from file

In [33]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords


class KaggleWord2VecUtility(object):
    """KaggleWord2VecUtility is a utility class for processing raw HTML text into segments for further learning"""

    @staticmethod
    def review_to_wordlist( review, remove_stopwords=False ):
        # Function to convert a document to a sequence of words,
        # optionally removing stop words.  Returns a list of words.
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text()
        #
        # 2. Remove non-letters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
        #
        # 3. Convert words to lower case and split them
        words = review_text.lower().split()
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]
        #
        # 5. Return a list of words
        return(words)
    
    @staticmethod
    def review_to_words( review, remove_stopwords=False ):
        # Function to convert a raw review to a string of words
        # The input is a single string (a raw movie review), and 
        # the output is a single string (a preprocessed movie review)
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text() 
        #
        # 2. Remove non-letters        
        review_text = re.sub("[^a-zA-Z]", " ", review_text) 
        #
        # 3. Convert to lower case, split into individual words
        words = review_text.lower().split()                             
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]   
        #
        # 6. Join the words back into one string separated by space, 
        # and return the result.
        return( " ".join( words ))   

    # Define a function to split a review into parsed sentences
    @staticmethod
    def review_to_sentences( review, tokenizer, remove_stopwords=False ):
        # Function to split a review into parsed sentences. Returns a
        # list of sentences, where each sentence is a list of words
        #
        # 1. Use the NLTK tokenizer to split the paragraph into sentences
        raw_sentences = tokenizer.tokenize(review.decode('utf8').strip())
        #
        # 2. Loop over each sentence
        sentences = []
        for raw_sentence in raw_sentences:
            # If a sentence is empty, skip it
            if len(raw_sentence) > 0:
                # Otherwise, call review_to_wordlist to get a list of words
                sentences.append( KaggleWord2VecUtility.review_to_wordlist( raw_sentence, \
                  remove_stopwords ))
        #
        # Return the list of sentences (each sentence is a list of words,
        # so this returns a list of lists
        return sentences